In [ ]:
!pip install openai -q

In [ ]:
import openai
openai.api_key = "YOUR_API_KEY"

In [ ]:
import xml.etree.ElementTree as ET

def read_xml_to_list_of_dicts(file_name):
    tree = ET.parse(file_name)
    root = tree.getroot()

    data = []

    for pair in root.findall('pair'):
        pair_data = {}
        pair_data['id'] = pair.get('id')

        t1 = pair.find('t1')
        if t1 is not None:
            pair_data['t1'] = t1.text.strip()

        t2 = pair.find('t2')
        if t2 is not None:
            pair_data['t2'] = t2.text.strip()

        data.append(pair_data)

    return data

file_name = "TestData_en.xml"
data_list = read_xml_to_list_of_dicts(file_name)
print(data_list)

In [ ]:
from tqdm import tqdm
def ask_gpt4(question, context):
    prompt = f"{context}\n\nQuestion: {question}\nAnswer (Y or N), no explan:"
    while True:
      try:
        response=openai.ChatCompletion.create(
            model="gpt-4-0314", 
            n=1,
            messages=[
                  {"role": "user", "content":prompt}
            ]
        )
        break
      except Exception as e:
        print(f"Error when calling API: {e}")

    answer = response.choices[0].message.content
    return answer

output_file = "GPT-4_output.txt"

with open(output_file, "w") as f:
    for pair in tqdm(data_list):
        t1 = pair['t1']
        t2 = pair['t2']
        id_ = pair['id']

        answer = ask_gpt4(t2, t1)
        f.write(f"{id_} {answer} GPT4\n")